## 正确运算符重载

- [运算符重载](#运算符重载)
- .[运算符类型](#运算符类型)
    - .[unary operator](#unary-operator)
    - .[infix operator](#infix-operator)
        - .[中缀运算符列表](#list)
- .[正确重载向量加法运算符](#向量加法运算符)
    - .[e.g. Array](#monkey)
- .[正确重载向量乘法运算符](#mul)
- .[众多比较运算符](#cmp)
- .[增量赋值运算符](#增量)   
    - .[不可变类型](#val)

#### 运算符重载

- built-in类型不支持
- 不能新建运算符
- 部分运算符不能重载,比如 and, is, or, not

#### 运算符类型

按照operands主要分为
- unary operator 一元运算符,比如-(neg), +(pos), ~(invert)
- infix operator 中缀运算符,比如+-(向量加减法), == > <等(比较运算符)和+= -=(增量赋值运算符)

In [1]:
import import_ipynb
from chapter_10 import Array

importing Jupyter notebook from chapter_10.ipynb


#### unary operator

In [2]:
a = Array([1, 2])

In [3]:
from functools import partial
import math

def power(base, n):
    return base ** n

square = partial(power, n=2)

def array_abs(self):
    return math.sqrt(sum(map(square, self)))


def _neg(self):
    return Array(-x for x in self)

In [4]:
Array.__abs__ = array_abs


abs(a)

2.23606797749979

In [5]:
#-a # TypeError: bad operand type for unary -: 'Array'

In [6]:
Array.__neg__ = _neg

-a

Vector([-1.0, -2.0])

#### infix operator

中缀运算符,二元运算符,运算符位于两个operands中间

####  中缀运算符列表 <a id='list'></a>


| 运算符 | dunder method| 解释|
| --- | ---- | ---|
| + | __add__ | 加法|
| - | __sub__ | 减法|
| * | __mul__| |
| / | __truediv__ ||
| // | __floordiv__ | |
| % | __mod__ | |
| divmod | __divmod__ | 返回整除的商和模组成的元组|
| ** pow | __pow__ | 取幂|
| @ | __matmul__| |
| & | __and__ | |
| \| | __or__ | |
| ^ | __xor__ | |
| << | __lshift__ | 按位左移 |
| >> | __rshift__ | 按位右移 | 

#### 向量加法运算符

#### 猴子补丁来定义__add__方法 <a id='monkey'></a>

In [7]:
from itertools import zip_longest

def vector_add(self, other):
    pairs = zip_longest(self, other, fillvalue=0.0)
    return Array(a+b for a, b in pairs)   


Array.__add__ = vector_add

In [8]:
a = Array([1, 2])

b = Array([2, 3, 4])
a + b

Vector([3.0, 5.0, 4.0])

In [9]:
a + [1.1, 2.2]

Vector([2.1, 4.2])

#### radd方法

right加法

In [10]:
from chapter_9 import Array2D
c = Array2D(3, 4)

c + b # Array2D的__add__方法没有实现

importing Jupyter notebook from chapter_9.ipynb
(3.0, 4.0)
3.0
4.0
There are only two elements


TypeError: unsupported operand type(s) for +: 'Array2D' and 'Array'

In [11]:
def vector_radd(self, other):
    return self + other

Array.__radd__ = vector_add
c + b  # 调用 b的__radd__方法

Vector([5.0, 7.0, 4.0])

#### 安全重载加法

如果类型出错时,报错信息复杂无效

In [12]:
a + 1

TypeError: zip_longest argument #2 must support iteration

In [13]:
def add_safe(self, other):
    """"""
    try:
        pairs = zip_longest(self, other, fillvalue=0.0)
        return Array(pairs)
    except TypeError:
        return NotImplemented
        
        
Array.__add__ = add_safe
        
        
a + 1

TypeError: unsupported operand type(s) for +: 'Array' and 'int'

### 乘法 <a id='mul'></a>

$ \text{__mul__}, \text{__rmul__} $

In [14]:
def mul(self, other):
    import numbers
    if isinstance(other, numbers.Real):
        return Array(x * other for x in self)
    
    
Array.__mul__ = mul 

a * 3

Vector([3.0, 6.0])

### 众多比较运算符 <a id='cmp'></a>

| 运算符 |dunder method | 说明|
| --- | --- | ----|
| == | __eq__ | |
| != | __ne__ | |
| > | __gt__ | |
| >= | __ge__ | |
| < | __lt__ | |
| <= | __le__ | |


In [15]:
def array_eq(self, other):
    if isinstance(other, Array):
        return len(other) == len(self) and all(x == y for x, y in zip(self, other))
    else:
        return NotImplemented
    
    
Array.__eq__ = array_eq

a == Array([1, 2])

True

In [16]:
a == 2  # 调用__ne__方法,然后比较id

False

### 增量赋值 <a id='增量'></a>

+=始终会新建对象 <a id='val'></a>

In [17]:
tmp = 4
id(tmp)

4361309456

In [18]:
tmp -= 1
id(tmp)

4361309424

In [25]:
def array_iadd(self, other):
    if isinstance(other, Array) and len(other) == len(self):
        return Array(x + y for x, y in zip(self, other))
    else:
        return NotImplemented
    
Array.__iadd__ = array_iadd
a1 = Array([1.2, 1.3])
a1_alias = a1
print(id(a1), id(a1_alias))

a1 += a1 # 创建新的对象
a1

140548017497648 140548017497648


Vector([2.4, 2.6])

In [26]:
a += 2

TypeError: unsupported operand type(s) for +=: 'Array' and 'int'

In [27]:
print(id(a1), id(a1_alias))

140548017496528 140548017497648
